I want to use the rasterio library to mask the raster file using shapefile

In [1]:
import rasterio
from rasterio.features import geometry_mask
import geopandas as gpd
from rasterio.mask import mask

Create a python environment, download the rasterio and geopandas packages, introduce the packages
link:https://rasterio.readthedocs.io/en/stable/topics/masking-by-shapefile.html

In [2]:
with rasterio.open(r'C:\Users\31619\Desktop\CLCD_v01_2006_albert_province\CLCD_v01_2018_albert_shandong.tif') as src:
    raster_data = src.read(1)

Open the raster file and read the raster data. Here the path is represented as a raw string because it contains special characters such as spaces.

In [3]:
print(raster_data)

Verify that it contains raster data

In [4]:
meta = src.meta.copy()

Get the metadata information of a raster file

In [5]:
shapefile = gpd.read_file(r'C:\Users\31619\Desktop\CLCD_v01_2006_albert_province\QINGDAOCITY.shp')
print(shapefile.crs)

Open the shapefile,Check the coordinate system

In [6]:
shapefile.head()

check the content of shapefile

In [7]:
shapefile_geojson = shapefile.__geo_interface__

Convert Shapefile to GeoJSON format

mask = geometry_mask(shapefile.geometry, out_shape=raster_data.shape, transform=src.transform, invert=True)
if not mask.any():
    raise ValueError('Input shapes do not overlap raster.')

Check if the geometry in the shapefile has overlapping parts with the raster data. If there are no overlapping parts, a 'ValueError' will be raised.

In [9]:
masked_data, masked_transform = mask(dataset=src, shapes=[shapefile_geojson], crop=True)

Masking operations using raster data and geometric information from Shapefile

In [10]:
meta.update({'transform': masked_transform, 'height': masked_data.shape[1], 'width': masked_data.shape[2]})

Update metadata information for raster data

In [11]:
with rasterio.open(r'C:\Users\31619\Desktop\CLCD_v01_2006_albert_province/output_raster.tif', 'w', **meta) as dst:
    dst.write(masked_data)

Save the masked raster data to a new file

In [13]:
print("Masked raster file saved successfully.")